<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_NER_Training_(transformer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy-transformers
!python -m spacy download zh_core_web_trf
!pip install spacy-pkuseg

2021-08-11 00:58:26.963830: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 417.4 MB 17 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_trf')


In [2]:
sentences = [
  '恆指上升3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。',
  '中電在假期前做好帶動，恒指單日暴升961.22點。',
  '受美股收跌拖累，匯控、和記跌超過2%。',
  '匯控下跌2%。',
  '日本、新加坡、澳洲股市單日暴升961.22點。'
]

TRAIN_DATA = [
  (
    '現時恒指已跌穿重要支持位26500點來看，港股繼續尋底的機會是頗高的。',
    {
        'entities':
        [
         (2,4,'FIN'),(9,12,'FIN'),(21,23,'FIN'),(25,27,'FIN')
        ]
    }
  ),
  (
    '恆生指數和創業板指數。',
    {
      'entities': 
      [
        (0,4,'FIN'),(5,10,'FIN')          
      ]
    }
  ),  
  (
    '中電、匯控、和黃8月暴升8.5%。',
    {
      'entities': 
      [
        (0,2,'STOCK'),(3,5,'STOCK'),(6,8,'STOCK'),(10,12,'FIN')         
      ]
    }
  ),
  (
    '恆指小雙底9月盼反彈。中電、匯控、和黃8月齊齊跌8.5%金融海嘯後最差。',
    {
      'entities': 
      [
        (0,2,'FIN'),(2,5,'FIN'),(8,10,'FIN'),(11,13,'STOCK'),(14,16,'STOCK'),(17,19,'STOCK')      
      ]
    }
  ),     
  (
    '匯控(00005)將於下周一（2日）公布2021年度中期業績。',
    {
        'entities':
        [
          (0,2,'STOCK'),(26,30,'FIN')
        ]
    }
  ),
  (
    '查看香港最新的股市報價、記錄、新聞及其他重要資訊，助你賣買股票及投資。',
   {
      'entities':
      [
        (2,4,'GPE'),(7,9,'FIN'),(9,11,'FIN')
      ]
   }
  ),
  (
    '匯控低開4%。港股收市跌。',
    {
        'entities':
        [
         (0,2,'STOCK'),(7,9,'FIN')
        ]
    }
  ),
  (
    '港股高開173點。',
   {
       'entities': [(0,2,'FIN'),(2,4,'FIN')]
   }
  ),
  (
    '報道引述匯控內部電郵稱',{'entities':[(4,6,'STOCK')]}
  ),
  (
    '匯控將在第四季開始進行股份回購',{'entities':[(0,2,'STOCK')]}
  ),
    (
    '花旗發表研究報告，下調匯控（0005）每股盈利預測',{'entities':[(11,13,'STOCK')]}
  ),
    (
    '匯控將恢復派息',{'entities':[(0,2,'STOCK')]}
  )
]

In [3]:
import spacy
from spacy import displacy

nlp = spacy.load('zh_core_web_trf')
# nlp.tokenizer.pkuseg_update_user_dict(['匯控','和記','中電'])

for s in sentences:
  doc = nlp(s)
  displacy.render(doc,jupyter=True,style="ent")

  # for ent in doc.ents:
  #   print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [11]:
import random
from spacy.training.example import Example
from spacy.util import minibatch, compounding

# source_nlp = spacy.load('zh_core_web_trf')
# print(source_nlp.pipe_names)
# nlp = spacy.blank("zh")
# nlp.add_pipe("transformer",source=source_nlp)
# nlp.add_pipe("ner",source=source_nlp)

ner = nlp.get_pipe('ner')
ner.add_label('STOCK')
ner.add_label('FIN')

nlp.tokenizer.pkuseg_update_user_dict(['匯控','和記'])

# https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

epoch = 5
batch_size = compounding(2.0, 12.0, 1.001)

# List of pipes you want to train
pipe_exceptions = ["ner", "transformer"]
# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):
  optimizer = nlp.resume_training()
  for itn in range(epoch):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for batch in minibatch(TRAIN_DATA, size=batch_size):
      for text, annotations in batch:
          doc = nlp.make_doc(text)
          example = Example.from_dict(doc, annotations)
          nlp.update([example], sgd = optimizer, drop=0.4, losses=losses)
      print (f'epoch: {itn}, losses: {losses}')

# nlp.to_disk('saved_model')

print(f'NER labels: {ner.labels}')

# https://github.com/explosion/spaCy/discussions/7576
# https://reposhub.com/python/deep-learning/explosion-spacy-transformers.html

epoch: 0, losses: {'transformer': 194.7398374080658, 'ner': 78.76234993144223}
epoch: 1, losses: {'transformer': 133.7417841553688, 'ner': 81.56808013751497}
epoch: 2, losses: {'transformer': 133.6923017501831, 'ner': 80.73369705924891}
epoch: 3, losses: {'transformer': 128.20902103185654, 'ner': 80.68752993702734}
epoch: 4, losses: {'transformer': 172.34345126152039, 'ner': 83.02298830614043}
NER labels: ('CARDINAL', 'DATE', 'EVENT', 'FAC', 'FIN', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'STOCK', 'TIME', 'WORK_OF_ART')


In [12]:
# nlp = spacy.load('saved_model')

for s in sentences:
  doc = nlp(s)
  displacy.render(doc,jupyter=True,style="ent")
